# 2. データパイプライン

In [ ]:
### 2.1 トークナイザー設定
# Model and tokenizer preparation
model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    token=os.environ["HUGGINGFACE_TOKEN"],  
    trust_remote_code=True
)


# Add special tokens to tokenizer
tokenizer.add_special_tokens({
    'additional_special_tokens': [
        '。', '、', '！', '？',  # Punctuation marks
    ]
})


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

このコードについて、ソクラテス式チャットボットの文脈で説明させていただきます。

### 1. モデルとトークナイザーの準備

```python
model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    token=os.environ["HUGGINGFACE_TOKEN"],  
    trust_remote_code=True
)
```

このコードは、テキストを処理するための「トークナイザー」を設定しています。トークナイザーとは、文章を機械が理解できる形に変換するツールです。

具体例を挙げると：
- 入力：「なぜそう考えるのですか？」
- トークナイザーの処理：「なぜ」「そう」「考える」「の」「です」「か」「？」のように分割

ここでは、日本語に特化した Google の Gemma モデルのトークナイザーを使用しています。

### 2. 特殊トークンの追加

```python
tokenizer.add_special_tokens({
    'additional_special_tokens': [
        '。', '、', '！', '？',  # Punctuation marks
    ]
})
```

この部分は、日本語の重要な句読点をトークナイザーに特別なトークンとして認識させています。

ソクラテス式対話での具体例：
```
入力文：「その考えは興味深いですね。なぜそう思ったのですか？」

通常の処理：
「その」「考え」「は」「興味」「深い」「です」「ね」「。」「なぜ」「そう」「思った」「の」「です」「か」「？」

特殊トークン追加により：
- 「。」と「？」が特別な意味を持つトークンとして認識される
- これにより、文の区切りや疑問文の特徴をAIがより正確に理解できる
```

この設定は特に重要です。なぜなら：
1. ソクラテス式対話では、質問文（「？」）が頻繁に使用される
2. 文の区切り（「。」）で、一つの問いかけや考えが完結する
3. 「！」は驚きや気づきを表現する際に使用される
4. 「、」は、考えを整理して話を進める際の区切りとして重要

これらの句読点を特別に認識することで、AIがより自然なソクラテス式の対話を生成できるようになります。

        
</div>
    </details>
</div>


In [ ]:

### 2.2 データセット準備
def validate_message_format(message):
    """Validate message format"""
    if not isinstance(message, dict):
        return False
    if 'role' not in message or 'content' not in message:
        return False
    if message['role'] not in ['user', 'model']:
        return False
    if not isinstance(message['content'], str):
        return False
    return True

def prepare_dataset():
    conversations = []
    
    try:
        with open(DIALOGUE_JSON_PATH, 'r', encoding='utf-8') as f:
            dialogue_data = json.load(f)
            
        for dialogue in dialogue_data:
            messages = dialogue.get('messages', [])
            
            # Validate message format
            if not all(validate_message_format(msg) for msg in messages):
                logging.warning(f"Skipped dialogue due to invalid message format")
                continue
                
            # Build conversation checking user->model sequence
            current_conversation = []
            valid_sequence = True
            
            for i in range(0, len(messages)-1, 2):
                if (i+1 < len(messages) and 
                    messages[i]['role'] == 'user' and 
                    messages[i+1]['role'] == 'model'):
                    current_conversation.extend([messages[i], messages[i+1]])
                else:
                    valid_sequence = False
                    break
            
            # Add only valid conversations
            if valid_sequence and current_conversation:
                # Apply Gemma chat template
                formatted_text = tokenizer.apply_chat_template(
                    current_conversation,
                    tokenize=False,
                    add_generation_prompt=True
                )
                
                # Check token count
                tokens = tokenizer.encode(formatted_text)
                if len(tokens) <= MAX_SEQUENCE_LENGTH:
                    conversations.append({"text": formatted_text})
                else:
                    logging.warning(f"Skipped conversation due to length: {len(tokens)} tokens")
            
    except Exception as e:
        logging.error(f"Error processing dialogue file: {str(e)}")
        raise
    
    if not conversations:
        raise ValueError("No valid conversations found in the dialogue file")
        
    logging.info(f"Processed {len(conversations)} valid conversations")
    return Dataset.from_list(conversations)
dataset = prepare_dataset()

# Check dataset structure
print("Dataset structure:")
print(dataset[0])  # Display first element
print("\nDataset features:")
print(dataset.features)
dataset = dataset.select(range(len(dataset))).shuffle(seed=42)



<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

このコードについて、ソクラテス式チャットボットの文脈で説明させていただきます。

### 1. メッセージフォーマットの検証
```python
def validate_message_format(message):
    """Validate message format"""
    if not isinstance(message, dict):
        return False
    if 'role' not in message or 'content' not in message:
        return False
    if message['role'] not in ['user', 'model']:
        return False
    if not isinstance(message['content'], str):
        return False
    return True
```

この関数は、対話データが正しい形式かどうかをチェックします。

具体例：
```python
# 正しい形式：
{
    'role': 'user',
    'content': 'なぜ哲学は重要だと思いますか？'
}

# 間違った形式：
{
    'speaker': 'user',  # 'role'ではないのでエラー
    'text': '哲学について考えましょう'  # 'content'ではないのでエラー
}
```

### 2. データセットの準備
```python
def prepare_dataset():
    conversations = []
    
    try:
        with open(DIALOGUE_JSON_PATH, 'r', encoding='utf-8') as f:
            dialogue_data = json.load(f)
```

この部分は対話データをJSONファイルから読み込みます。

### 3. 対話の検証とフォーマット
```python
for dialogue in dialogue_data:
    messages = dialogue.get('messages', [])
    
    # メッセージ形式の検証
    if not all(validate_message_format(msg) for msg in messages):
        logging.warning(f"Skipped dialogue due to invalid message format")
        continue
```

各対話が以下のような形式になっているか確認します：

```python
# 正しい対話の例：
{
    "messages": [
        {
            "role": "user",
            "content": "幸せとは何だと思いますか？"
        },
        {
            "role": "model",
            "content": "その質問は興味深いですね。あなたにとって幸せとは何でしょうか？"
        }
    ]
}
```

### 4. 対話順序の確認
```python
for i in range(0, len(messages)-1, 2):
    if (i+1 < len(messages) and 
        messages[i]['role'] == 'user' and 
        messages[i+1]['role'] == 'model'):
        current_conversation.extend([messages[i], messages[i+1]])
```

この部分は、対話が「ユーザーの質問→モデルの応答」という順序になっているか確認します。

例：
```python
# 正しい順序：
1. ユーザー: "知識とは何でしょうか？"
2. モデル: "その問いについて一緒に考えてみましょう。あなたは知識をどのように定義しますか？"
3. ユーザー: "自分が確実に理解していることだと思います"
4. モデル: "なるほど。では、確実な理解とは何を指すのでしょうか？"

# 間違った順序（スキップされる）：
1. モデル: "こんにちは"
2. ユーザー: "こんにちは"
```

### 5. 対話の長さチェックと保存
```python
tokens = tokenizer.encode(formatted_text)
if len(tokens) <= MAX_SEQUENCE_LENGTH:
    conversations.append({"text": formatted_text})
else:
    logging.warning(f"Skipped conversation due to length: {len(tokens)} tokens")
```

この部分は、対話が適切な長さ（MAX_SEQUENCE_LENGTH以下）かどうかをチェックします。長すぎる対話は学習データから除外されます。

最後に、検証を通過した対話データをシャッフルして、学習用データセットとして準備します：
```python
dataset = dataset.select(range(len(dataset))).shuffle(seed=42)
```

これにより、ソクラテス式の対話パターンを学習するための質の高いデータセットが準備されます。

        
</div>
    </details>
</div>


In [ ]:
### 2.3 データ前処理と検証

def tokenize_function(examples):
    result = tokenizer(
        examples['text'],
        truncation=True,
        max_length=TOKENIZE_MAX_LENGTH,      # 256 から TOKENIZE_MAX_LENGTH に変更
        padding='max_length',
        add_special_tokens=True,
        return_tensors=None,
    )
    return result

# Add dataset preprocessing
def preprocess_function(examples):
    # Pattern definitions
    end_patterns = [
        "だろうか", "ではないか", "のではないか", "かね",
        "なるほど", "興味深い", "考えてみよう"
    ]
    
    # Conjunction patterns
    conjunctions = [
        "しかし", "だから", "それでは", "すなわち",
        "たとえば", "つまり", "ならば", "もし"
    ]
    
    # Get tokenized texts
    texts = tokenizer.batch_decode(examples['input_ids'])
    new_attention_masks = []
    
    for text, mask in zip(texts, examples['attention_mask']):
        if not isinstance(mask, list):
            mask = mask.tolist()
        
        # Create new attention mask (base value 0.8)
        new_mask = [0.8] * len(mask)
        
        # Split into sentences
        sentences = text.split('。')
        current_pos = 0
        
        for sentence in sentences:
            if not sentence.strip():
                continue
                
            # Detect and emphasize end patterns
            for pattern in end_patterns:
                if pattern in sentence:
                    # Locate pattern position
                    pattern_tokens = tokenizer.encode(pattern, add_special_tokens=False)
                    pattern_len = len(pattern_tokens)
                    
                    # Emphasize tokens containing pattern and surrounding tokens
                    pattern_start = current_pos + len(tokenizer.encode(sentence, add_special_tokens=False)) - pattern_len
                    for i in range(max(0, pattern_start - 2), min(len(mask), pattern_start + pattern_len + 2)):
                        new_mask[i] = 1.0  # Maximum attention for pattern parts
            
            # Detect and emphasize conjunctions
            for conj in conjunctions:
                if conj in sentence:
                    # Locate conjunction position
                    conj_tokens = tokenizer.encode(conj, add_special_tokens=False)
                    conj_pos = len(tokenizer.encode(sentence.split(conj)[0], add_special_tokens=False))
                    
                    # Emphasize tokens before and after conjunction (slightly lower)
                    for i in range(max(0, current_pos + conj_pos - 1), 
                                 min(len(mask), current_pos + conj_pos + len(conj_tokens) + 1)):
                        new_mask[i] = 0.9
            
            # Emphasize punctuation marks
            for i, char in enumerate(sentence):
                if char in '、。！？':
                    # Locate punctuation position
                    punct_pos = len(tokenizer.encode(sentence[:i], add_special_tokens=False))
                    # Emphasize tokens around punctuation
                    for j in range(max(0, current_pos + punct_pos - 1),
                                 min(len(mask), current_pos + punct_pos + 2)):
                        new_mask[j] = 0.95
            
            # Update position for next sentence
            current_pos += len(tokenizer.encode(sentence + '。', add_special_tokens=False))
        
        # Set special token masks to 1.0
        if tokenizer.bos_token_id is not None:
            new_mask[0] = 1.0  # BOS token
        if tokenizer.eos_token_id is not None:
            new_mask[-1] = 1.0  # EOS token
            
        new_attention_masks.append(new_mask)

    examples['attention_mask'] = new_attention_masks
    return examples




<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



このコードについて、ソクラテス式チャットボットの文脈で説明させていただきます。

### 1. トークン化関数
```python
def tokenize_function(examples):
    result = tokenizer(
        examples['text'],
        truncation=True,
        max_length=TOKENIZE_MAX_LENGTH,
        padding='max_length',
        add_special_tokens=True,
        return_tensors=None,
    )
    return result
```

これは対話テキストをトークン（機械が理解できる単位）に変換する関数です。

例：
```python
入力テキスト：
"なぜそう考えるのですか？"

トークン化後：
[BOS]  # 文章開始トークン
"なぜ" "そう" "考える" "の" "です" "か" "？"
[EOS]  # 文章終了トークン
```

### 2. 前処理関数
```python
def preprocess_function(examples):
    # パターン定義
    end_patterns = [
        "だろうか", "ではないか", "のではないか", "かね",
        "なるほど", "興味深い", "考えてみよう"
    ]
    
    conjunctions = [
        "しかし", "だから", "それでは", "すなわち",
        "たとえば", "つまり", "ならば", "もし"
    ]
```

この関数は、ソクラテス式対話に特徴的な表現パターンに注目して、それらにより強い注意を払うように設定します。

### 3. アテンションマスクの作成
```python
# Create new attention mask (base value 0.8)
new_mask = [0.8] * len(mask)
```

アテンションマスクとは、モデルがテキストのどの部分により注目すべきかを示す値です：
- 基本値: 0.8（通常の注意度）
- 1.0（最大注意）
- 0.9（やや強い注意）
- 0.95（強い注意）

### 4. 重要表現の強調
例えば以下のような対話で：
```
「なぜそう考えるのでしょうか？ なるほど、興味深い視点ですね。しかし、それについてもう少し考えてみましょう。」
```

各部分の注目度が以下のように設定されます：

1. 文末パターン（1.0の最大注意）：
```python
# "なるほど", "興味深い", "考えてみよう" など
pattern_start = current_pos + len(tokenizer.encode(sentence, add_special_tokens=False)) - pattern_len
for i in range(max(0, pattern_start - 2), min(len(mask), pattern_start + pattern_len + 2)):
    new_mask[i] = 1.0
```

2. 接続詞（0.9のやや強い注意）：
```python
# "しかし", "だから", "すなわち" など
for i in range(max(0, current_pos + conj_pos - 1), 
             min(len(mask), current_pos + conj_pos + len(conj_tokens) + 1)):
    new_mask[i] = 0.9
```

3. 句読点（0.95の強い注意）：
```python
# "、", "。", "！", "？"
for j in range(max(0, current_pos + punct_pos - 1),
             min(len(mask), current_pos + punct_pos + 2)):
    new_mask[j] = 0.95
```

この設定により、モデルは：
1. ソクラテス式の問いかけ表現（「だろうか」「ではないか」など）
2. 論理的な接続（「しかし」「つまり」など）
3. 文の区切り（句読点）

に特に注意を払いながら学習を行い、より自然なソクラテス式の対話を生成できるようになります。

        
</div>
    </details>
</div>


In [ ]:

### 2.4 データセット最適化
# Optimize dataset processing
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    batch_size=16,
    num_proc=2,
    load_from_cache_file=True,
    desc="Tokenizing datasets",
    remove_columns=dataset.column_names,
)

# Add dataset validation
def validate_dataset(dataset):
    # Check first element
    first_item = dataset[0]
    print("Validated first item structure:")
    print(f"Keys: {first_item.keys()}")
    print(f"input_ids type: {type(first_item['input_ids'])}")
    print(f"input_ids length: {len(first_item['input_ids'])}")
    return dataset



tokenized_dataset = validate_dataset(tokenized_dataset)


tokenized_dataset = tokenized_dataset.map(
    preprocess_function,
    batched=True,
    desc="Applying attention masking"
)




<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



このコードについて、ソクラテス式チャットボットの文脈で説明させていただきます。

### 1. データセットのトークン化処理
```python
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    batch_size=16,
    num_proc=2,
    load_from_cache_file=True,
    desc="Tokenizing datasets",
    remove_columns=dataset.column_names,
)
```

これは対話データを効率的にトークン化する処理です。

具体的な設定：
- `batched=True, batch_size=16`: 16個の対話を一度に処理
- `num_proc=2`: 2つのプロセスで並列処理
- `load_from_cache_file=True`: 処理結果をキャッシュして再利用可能に

例えば：
```python
# 入力データ
[
    "哲学とは何でしょうか？",
    "その質問は本質を突いていますね。あなたは哲学をどのように考えますか？",
    # ... 他の対話 ...
]

# トークン化後
[
    [1, 345, 67, 89, 2],  # 数値化されたトークン
    [1, 234, 56, 78, 90, 2],
    # ... 他の対話のトークン ...
]
```

### 2. データセットの検証
```python
def validate_dataset(dataset):
    first_item = dataset[0]
    print("Validated first item structure:")
    print(f"Keys: {first_item.keys()}")
    print(f"input_ids type: {type(first_item['input_ids'])}")
    print(f"input_ids length: {len(first_item['input_ids'])}")
    return dataset
```

この関数は、データセットが正しい形式になっているか確認します。

出力例：
```python
Validated first item structure:
Keys: dict_keys(['input_ids', 'attention_mask'])
input_ids type: <class 'list'>
input_ids length: 512  # 設定された最大長
```

### 3. アテンションマスクの適用
```python
tokenized_dataset = tokenized_dataset.map(
    preprocess_function,
    batched=True,
    desc="Applying attention masking"
)
```

これは先ほど説明した重み付けを対話データに適用します。

例えば、以下のような対話：
```python
"なぜそう考えるのでしょうか？（注目度1.0）
なるほど（注目度1.0）、
興味深い（注目度1.0）視点ですね。
しかし（注目度0.9）、
それについてもう少し考えてみましょう（注目度1.0）。"
```

このように、ソクラテス式対話の重要な要素（問いかけ、応答、接続詞など）に適切な注目度が設定され、モデルがこれらのパターンをより効果的に学習できるようになります。

この最適化により：
1. 効率的なデータ処理（並列処理、キャッシュ利用）
2. データ形式の検証（エラー防止）
3. ソクラテス式対話に特化した重み付け

が実現され、質の高い学習データセットが準備されます。

        
</div>
    </details>
</div>
